In [ ]:
# Imports 
from plantcv import plantcv as pcv
import sys, traceback, cv2, argparse, pathlib, os, matplotlib 
import numpy as np
from datetime import datetime
%matplotlib notebook


In [ ]:
pcv.__version__ 
# want to be on v4.x so that we can use the new parallelization and NIR+RBG to get NDVI spectral combination 
# Shouldd be something like 3.14.1+781.ge53ea6e

# Testing purposes
Create a list of test images. I have 25 VIS images per experimental code, and a corresponding NIR image within the testing directory. Should just be z1 and SV images, includes both 0 and 90 rotation viewpoints. 

In [ ]:
path = "./imgs/testing/"
dir_list = os.path.expanduser(path)
dir_list = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(path)) for f in fn]

img_list = [] 
for file in dir_list:
    if ".png" == pathlib.Path(file).suffix and "VIS" in file and "SV" in file:
        img_list.append(file)

# Start

In [ ]:
# Used this code to loop through replicates.
i=98

In [ ]:
i += 1
print(i)

In [ ]:
from plantcv.parallel import WorkflowInputs
# Set input variables
args = WorkflowInputs(images=[img_list[i]],
                      names="image",
                      result="plantcv_results.csv",
                      debug="plot")

# Set variables
pcv.params.debug = args.debug

rgb_img, path, filename = pcv.readimage(filename=args.image)
print(path)


In [ ]:
path_components = os.path.normpath(path).split(os.path.sep)
date = datetime.fromisoformat(path_components[-2])

In [ ]:
# Set ROI location depending on timepoint 
roi_w = 900
if date <= datetime(2016, 5, 27):
    roi_h = 1520 
elif datetime(2016,5,27) < date <= datetime(2016,6,7):
    roi_h = 1460
elif datetime(2016,6,7) < date <= datetime(2016,8,20): # lt2 is annoying :( 
    roi_h = 1520
# elif datetime(2016,8,20) < date <= datetime(2016,8,20):
#     roi_h = 1340
# elif datetime(2016,8,20) < date <= datetime(2016,9,27):
#     roi_h = 1230
     
    
## Info about which indices correspond to which experiments 
# img_list_lt1a = img_list[:49] # End date 2016-05-27                       --> roi x=800, y=100, h=1520, w=900
# img_list_lt1b = img_list[50:99] # 2016-05-27 through 2016-06-06           -->     x=800, y=100, h=1460, w=900
# img_list_lt2 = img_list[100:149] 
# 2016-08-01 through 2016-08-20.                                            --> roi x=800, y=100, h=1520, w=900
    # inconsistent though UGH first few reps are lower and on 2016-08-20    -->     x=800, y=100, h=1340, w=900
# img_list_lt3 = img_list[150:199] # 2016-09-27 through 2016-10             -->      x=800, y=100, h=1230, w=900
    # Inconsistent again. First few reps are lower (before 10/07)

# img_list_lt4a = img_list[200:249] # 2016-11-28 through 2016-12-09         -->      x=800, y=100, h=1225, w=900
    # Segmentation issues, picking up lots of the side of pot 
    # Include empty pot??? mask the pot
# img_list_lt4b = img_list[250:299] # 2016-12-09 through 2016-12-20         -->      x=800, y=100, h=1225, w=900
    # Right side of pot also an issue for this run 

# img_list_lt5a = img_list[300:349] # 2017-01-06 through end of the month   -->      x=800, y=100, h=1510, w=900
    # Inconsistent pot height again UGH taller pot 2017-01-18 til end       -->      x=800, y=100, h=1330, w=900
    # Taller pot again 2017-01-21 through end but large plants cause issue  -->      x=800, y=100, h=1230, w=900
# img_list_lt5b = img_list[350:399] # 2017-02-03 through end of the month   -->.     x=800, y=100, h=1520, w=900
    # Taller pot again 2017-02-11 through 

In [ ]:
roi, roi_hierarchy= pcv.roi.rectangle(img=rgb_img, x=800, y=100, h=roi_h, w=900)

In [ ]:
pcv.params.text_size = 3
pcv.params.text_thickness = 3
colorspace_img = pcv.visualize.colorspaces(rgb_img=rgb_img, original_img=False)

#labeled_imgs = pcv.visualize.auto_threshold_methods(gray_img=s, grid_img=True, object_type="light")

In [ ]:
b = pcv.rgb2gray_lab(rgb_img=rgb_img, channel='b')


In [ ]:
b_thresh = pcv.threshold.binary(gray_img=b, threshold=132, max_value=255, object_type='light')


In [ ]:
#labeled_imgs = pcv.visualize.obj_sizes(img=rgb_img, mask=b_thresh)

In [ ]:
fill = pcv.fill(bin_img=b_thresh, size=100)
id_objects, obj_hierarchy = pcv.find_objects(img=rgb_img, mask=fill)

In [ ]:
roi_objects, roi_hier, kept_mask, obj_area = pcv.roi_objects(img=rgb_img, roi_contour=roi,
                                                              roi_hierarchy=roi_hierarchy,
                                                              object_contour=id_objects,
                                                              obj_hierarchy=obj_hierarchy,
                                                              roi_type='partial')

In [ ]:
mask = pcv.median_blur(gray_img=kept_mask, ksize=5)
clean_mask = pcv.fill(bin_img=mask, size=500)


In [ ]:
blended = pcv.visualize.overlay_two_imgs(img1=rgb_img, img2=clean_mask, alpha=0.4)

# STOP 🛑 ✋ 

In [ ]:
#if obj_area > 0: # include in parallel but skip for now 
id_objects, obj_hierarchy = pcv.find_objects(img=rgb_img, mask=clean_mask)
obj, mask = pcv.object_composition(img=rgb_img, contours=id_objects, hierarchy=obj_hierarchy)
# Find shape properties, output shape image (optional)
shape_imgs = pcv.analyze_object(img=rgb_img, obj=obj, mask=mask, label="default")
# Shape properties relative to user boundary line (optional)
boundary_img1 = pcv.analyze_bound_horizontal(img=rgb_img, obj=obj, mask=mask, line_position=1330, label="default")
# Determine color properties: Histograms, Color Slices, output color analyzed histogram (optional)
color_histogram = pcv.analyze_color(rgb_img=rgb_img, mask=mask, hist_plot_type=None, colorspaces="hsv", label="default")

# Pseudocolor the grayscale image
s = pcv.rgb2gray_hsv(rgb_img=rgb_img, channel='s')
pseudocolored_img = pcv.visualize.pseudocolor(gray_img=s, mask=mask, cmap='jet')

In [ ]:
nir_path = pcv.get_nir(path=path, filename=filename)
nir_img, path, img_filename = pcv.readimage(filename= nir_path, mode="native")

# Transform/warp the image 
warped_img, mat = pcv.transform.warp(img=rgb_img, refimg=nir_img,
                                     pts=[(495,435), (500,1126), (512,1797), (1990,440), (1985,1125), (1976,1795)],
                                     refpts=[(105,119), (105,313), (104,499), (525,119), (525,313), (527,499)],
                                     method='lmeds')

In [ ]:
# Use same points to warp the mask to the NIR image 
warped_mask, mat2 = pcv.transform.warp(img=mask, refimg=nir_img,
                                     pts=[(495,435), (500,1126), (512,1797), (1990,440), (1985,1125), (1976,1795)],
                                     refpts=[(105,119), (105,313), (104,499), (525,119), (525,313), (527,499)],
                                     method='lmeds')

In [ ]:
# Output here seems wrong.. usually one large peak? 
analysis_nir = pcv.analyze_nir_intensity(gray_img=nir_img, mask=warped_mask, bins=256, histplot=None, label="default")

In [ ]:
wvs1 = [480.0, 550.0, 670.0]
wvs2 = [800.0]
fused_img = pcv.image_fusion(img1=warped_img, img2=nir_img, wvs1=wvs1, wvs2=wvs2,
                         array_type="vis-nir_fusion")
ndvi = pcv.spectral_index.ndvi(fused_img)
# Pseudocolor the NDVI image
colmap = pcv.visualize.pseudocolor(gray_img=ndvi.array_data, mask=warped_mask, cmap="RdYlGn",
                                   min_value=-0.8, max_value=0.8)

In [ ]:
gdvi = pcv.spectral_index.gdvi(fused_img)
psri = pcv.spectral_index.psri(fused_img)
colmap1 = pcv.visualize.pseudocolor(gray_img=psri.array_data, mask=warped_mask, cmap="RdYlGn",
                                   min_value=-0.8, max_value=0.8)
colmap2 = pcv.visualize.pseudocolor(gray_img=gdvi.array_data, mask=warped_mask, cmap="RdYlGn",
                                   min_value=-0.8, max_value=0.8)
gdvi_array = np.histogram(gdvi.array_data, bins=256)

# Calculate histogram
gdvi_hyper = pcv.hyperspectral.analyze_index(index_array=gdvi, mask=warped_mask, bins=100, min_bin=-2, max_bin=2, label="default")
psri_hyper = pcv.hyperspectral.analyze_index(index_array=psri, mask=warped_mask, bins=100, min_bin=np.amin(psri.array_data), max_bin=np.amax(psri.array_data), label="PSRI array data")
ndvi_hyper = pcv.hyperspectral.analyze_index(index_array=ndvi, mask=warped_mask, bins=100, min_bin=-1, max_bin=1, label="NDVI array data")

pcv.outputs.save_results(filename=args.result)